# 📲 Backup WhatsApp Chat to Excel 

#### Imports

In [7]:
# Standard Python Libaries
import urllib.request
import os
import ssl

# Third Party Modules
import pandas as pd # pip install pandas openpyxl
from pushbullet import Pushbullet # pip install pushbullet.py

#### Get Download Link from Pushbullet

In [3]:
# API KEY PushBullet (https://www.pushbullet.com/)
API_KEY = "o.5WdONr814nt1QR4JlBzWfw3QWplLQgWZ"

In [4]:
# https://pypi.org/project/pushbullet.py/0.9.1/
pb = Pushbullet(API_KEY)

# List of pushed file
pushes = pb.get_pushes()

# Get latest file [First Element in List]
latest = pushes[0]

# Get Link to Chatfile
url = latest['file_url']
url

'https://dl3.pushbulletusercontent.com/jVdxwmeTAnSNAJf2oN9sm5AqpXhdXidS/WhatsApp%20Chat%20with%20Finances.txt'

#### Download Chatfile from Pushbullet(link)

In [11]:
# coding: utf-8
import urllib.request
import ssl


def main():
    ssl._create_default_https_context = ssl._create_unverified_context
    r = urllib.request.urlopen('https://google.com')
    print(r.status)
    print(r)


if __name__ == '__main__':
    main()

200


In [12]:
# Download Chatfile
url = latest['file_url']
file_path = "chat.txt"

urllib.request.urlretrieve(url, file_path)

('chat.txt', <http.client.HTTPMessage at 0x1a609d54b20>)

#### Read Data

In [13]:
# Read file by line by line
with open(file_path, mode='r', encoding="utf8") as f:
    data = f.readlines()

In [16]:
# Print Data
data

['18/02/2021, 2:51 pm - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.\n',
 '18/02/2021, 2:51 pm - You created group "Finances"\n',
 '18/02/2021, 2:52 pm - Kaustubh: Saumya owe kaustubh 10 for beer\n',
 '18/02/2021, 2:52 pm - Kaustubh: Kaustubh owe Saumya for ebs (44)\n',
 '18/02/2021, 2:57 pm - Saumya: Saumya owe kaustubh 16 for fries 10 for banana 20 for fruits\n',
 '18/02/2021, 2:57 pm - Saumya: Kaustubh owe saumya 45+14\n',
 '18/02/2021, 3:12 pm - Saumya: Kaustubh owes Saumya 72 + 15 for burger republic\n',
 '18/02/2021, 9:53 pm - Saumya: 97.35 for burrito kaustubh owes\n',
 '18/02/2021, 9:54 pm - Kaustubh: thankq\n',
 '19/02/2021, 8:27 pm - Saumya: Kaustubh owes 39 for subway\n',
 '20/02/2021, 12:13 am - Kaustubh: You owe me 60 for food ssp\n',
 '20/02/2021, 1:12 am - Kaustubh: I owe you for games 115\n',
 '21/02/2021, 8:07 pm - Saumya: You owe 50 for zeugma\n',
 '22/02/2021, 6:26 pm - Sau

In [20]:
# Data in list format
data[-1]

'01/10/2021, 5:53 pm - Saumya: uome 60 for mos\n'

#### Clean Up data

In [21]:
# FOUND ON GITHUB: https://gist.github.com/kwcooper/a21ba58272d3cdf26310cc02ee4b168f
# parse text, create list of lists structure & remove first whatsapp info message
dataset = data[1:]
cleaned_data = []
for line in dataset:
    # Check, whether it is a new line or not
    # If the following characters are in the line -> assumption it is NOT a new line
    if '/' in line and ':' in line and ',' in line and '-' in line:
        # grab the info and cut it out
        date = line.split(",")[0]
        line2 = line[len(date):]
        time = line2.split("-")[0][2:]
        line3 = line2[len(time):]
        name = line3.split(":")[0][4:]
        line4 = line3[len(name):]
        message = line4[6:-1] # strip newline charactor
        cleaned_data.append([date, time, name, message])
    
    # else, assumption -> new line. Append new line to previous 'message'
    else:
        new = cleaned_data[-1][-1] + " " + line
        cleaned_data[-1][-1] = new    

#### 🚩 Create Dataframe & Export it to Excel 

In [22]:
# Create the DataFrame
df = pd.DataFrame(cleaned_data, columns = ['Date', 'Time', 'Name', 'Message'])

In [25]:
# Save it to excel
df.to_excel('chat_history.xlsx', index=False)